# QG Simulation

* Fourier Feature Networks (FFN)
* Siren
* Modulated Siren (ModSiren)
* Multiplicative Filter Networks (MFN)
    * Fourier
    * Gabor

In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])
local = root.joinpath("experiments/qg")

# append to path
sys.path.append(str(root))
sys.path.append(str(local))

In [ ]:
from typing import Dict, Any, cast
import tabulate
from IPython.display import display, HTML

import numpy as np
import xarray as xr
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import ReLU

from tqdm.notebook import tqdm as tqdm
import os, imageio

from inr4ssh._src.models.mlp import MLP
from inr4ssh._src.models.activations import Swish
from inr4ssh._src.datamodules.qg import QGSimulation

import pytorch_lightning as pl
from inr4ssh._src.models.image import ImageModel
from inr4ssh._src.models.siren import Siren, SirenNet, Modulator, ModulatedSirenNet
from inr4ssh._src.models.mfn import FourierNet, GaborNet
from inr4ssh._src.models.activations import get_activation

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.utilities.argparse import add_argparse_args
from pytorch_lightning.loggers import WandbLogger

import hvplot.xarray

pl.seed_everything(123)

import matplotlib.pyplot as plt
import seaborn as sns
import wandb

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Config

In [ ]:
import config
from simple_parsing import ArgumentParser

In [ ]:
# initialize argparse
parser = ArgumentParser()

# add all experiment arguments
parser.add_arguments(config.Logging, dest="logging")
parser.add_arguments(config.DataDir, dest="data")
parser.add_arguments(config.PreProcess, dest="preprocess")
parser.add_arguments(config.Features, dest="features")
parser.add_arguments(config.TrainTestSplit, dest="traintest")
parser.add_arguments(config.DataLoader, dest="dataloader")
parser.add_arguments(config.Model, dest="model")
parser.add_arguments(config.Siren, dest="siren")
parser.add_arguments(config.MLP, dest="mlp")
parser.add_arguments(config.FFN, dest="ffn")
parser.add_arguments(config.ModulatedSiren, dest="modsiren")
parser.add_arguments(config.MFN, dest="mfn")
parser.add_arguments(config.Losses, dest="losses")
parser.add_arguments(config.Optimizer, dest="optimizer")
parser.add_arguments(config.LRScheduler, dest="lr_scheduler")
parser.add_arguments(config.Callbacks, dest="callbacks")
# parser.add_arguments(config.EvalData, dest="eval")
# parser.add_arguments(config.Metrics, dest="metrics")
# parser.add_arguments(config.Viz, dest="viz")

# parse args
args = parser.parse_args([])

In [ ]:
args.dataloader.batch_size = 4096  # 8192

In [ ]:
# args.data.data_dir = f"/Users/eman/.CMVolumes/cal1_workdir/data/qg_data/public/"
args.data.data_dir = f"/Volumes/EMANS_HDD/data/qg_sim/"
args.logging.log_dir = "~/code_projects/logs/"
args.logging.mode = "online"

args.traintest.noise = None

args.traintest.step_Nx = 1
args.traintest.step_Ny = 1
args.traintest.step_time = 1
args.traintest.missing_data = 0.5

args.model = "siren"
model_config = args.siren

## Logger

In [ ]:
from inr4ssh._src.io import simpleargs_2_ndict

log_options = args.logging

# params_dict = simpleargs_2_ndict(args)

wandb_logger = WandbLogger(
    config=args,
    mode=log_options.mode,
    project=log_options.project,
    entity=log_options.entity,
    dir=log_options.log_dir,
    resume=False,
)

## Data

In [ ]:
config.DataDir

In [ ]:
# from inr4ssh._src.data.qg import load_qg_data

# ds = load_qg_data(dm.data.data_dir)

# ds = ds.coarsen({"time": 2}, boundary="trim", coord_func="mean")
# ds

In [ ]:
dm = QGSimulation(
    data=args.data,
    preprocess=args.preprocess,
    traintest=args.traintest,
    features=args.features,
    dataloader=args.dataloader,
    # eval=args.eval
)

dm.setup()

In [ ]:
len(dm.ds_train), len(dm.ds_valid), len(dm.ds_test), len(dm.ds_predict)

In [ ]:
# import pandas as pd
# import numpy as np


# def array_2_da(coords, data, name="full_pred", coords_name: List[str]=["x", "y", "t"]):
#     return pd.DataFrame(np.hstack([coords, data]), columns=[coords_name]+[name]).set_index(
#         coords_name).to_xarray()

In [ ]:
xr_data = dm.create_xr_dataset("predict")
xr_data = xr.merge([xr_data, dm.create_xr_dataset("train")])

In [ ]:
# xr_data.train.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
# xr_data.predict.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
# data.obs.thin(time=2).plot.imshow(
#     col="time", robust=True, col_wrap=4, cmap="viridis",
# )

The input data is a coordinate vector, $\mathbf{x}_\phi$, of the image coordinates.

$$
\mathbf{x}_\phi \in \mathbb{R}^{D_\phi}
$$

where $D_\phi = [\text{x}, \text{y}]$. So we are interested in learning a function, $\boldsymbol{f}$, such that we can input a coordinate vector and output a scaler/vector value of the pixel value.

$$
\mathbf{u} = \boldsymbol{f}(\mathbf{x}_\phi; \boldsymbol{\theta})
$$

### Data Module


#### Train-Test Split

In this example, we are only taking every other pixel for training and validation. It is a very simple and well-defined problem which each of the neural networks should be able to solve. The final test image is the original full resolution image.

Notice how we have `131_072` points from training and validation and `262_144` for the testing. This is because we have *raveled* the image where each coordinate is a vector of `x,y`. So these are a lot of points...

In [ ]:
init = dm.ds_train[:32]
x_init, y_init = init
x_init.shape, y_init.shape

In [ ]:
x_init.min(), x_init.max()

### Optimizer

For this, we will use a simple adam optimizer with a `learning_rate` of 1e-4. From many studies, it appears that a lower learning rate works well with this methods because there is a lot of data. In addition, a bigger `batch_size` is also desireable. We will set the `num_epochs` to `1_000` which should be good enough for a single image. Obviously more epochs and a better learning rate scheduler would result in better results but this will be sufficient for this demo.

In [ ]:
num_epochs = 300
learning_rate = 1e-4

### Scheduler

<p align="center">
<img src="http://www.bdhammel.com/assets/learning-rate/resnet_loss.png" alt="drawing" width="300"/>
<figcaption align = "center">
  <b>Fig.1 - An example for learning rate reduction when the validation loss stagnates. Source: 
    <a href="http://www.bdhammel.com/assets/learning-rate/resnet_loss.png">Blog</a>
  </b>
  </figcaption>
</p>

We will use a simple learning rate scheduler - `reduce_lr_on_plateau`. This will automatically reduce the learning rate as the validation loss stagnates. It will ensure that we really squeeze out as much performance as possible from our models during the training procedure.We start with a (relatively) high `learning_rate` of `1e-4` so we will set the `patience` to 5 epochs. So if there is no change in with every epoch, we decrease the learning rate by a factor of `0.1`.

This is a rather crude (but effective) method but it tends to work well in some situations. A better method might be the `cosine_annealing` method or the `exponential_decay` method. See other [examples](https://www.kaggle.com/code/snnclsr/learning-rate-schedulers/).

### Loss

We are going with a very simple `loss` function: the *mean squared error* (MSE). This is given by:

$$
\mathcal{L}(\boldsymbol{\theta}) = \frac{1}{N} \sum_n^N \left( \mathbf{y}_n - \boldsymbol{f}_{\boldsymbol{\theta}}(\mathbf{x}_n) \right)^2
$$

We won't code this from scratch, we will just use the PyTorch function, `nn.MSELoss`, and we will use the `mean` reduction parameter.


### PSNR

We will also keep track of the signal to noise ratio (PSNR) which will give us an indication of how well we are learning.

$$
\text{PSNR}(\mathbf{x}) = - 10 \log (2 * \text{MSE}(\mathbf{x}))
$$

In [ ]:
results = dict()

## Experiment

In [ ]:
num_steps_per_epoch = len(dm.train_dataloader())

In [ ]:
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

In [ ]:
class ImageModel(pl.LightningModule):
    def __init__(self, model, optimizer: str = "adam", **kwargs):
        super().__init__()

        self.save_hyperparameters()
        self.model = model
        self.hyperparams = cast(Dict[str, Any], self.hparams)
        self.loss = nn.MSELoss(reduction="mean")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # output
        x, y = batch

        pred = self.forward(x)

        # loss function
        loss = self.loss(y, pred)

        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        # output
        x, y = batch

        pred = self.forward(x)

        # loss function
        loss = self.loss(y, pred)

        self.log("val_loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        # output
        x, y = batch

        pred = self.forward(x)

        # loss function
        loss = self.loss(y, pred)

        self.log("test_loss", loss)

        return loss

    def predict_step(self, batch, batch_idx):
        # output
        x, y = batch

        pred = self.forward(x)

        return pred

    def configure_optimizers(self):

        # configure optimizer
        if self.hyperparams.optimizer == "adam":
            optimizer = Adam(
                self.model.parameters(), lr=self.hyperparams.get("lr", 1e-4)
            )
        elif self.hyperparams.optimizer == "lbfgs":
            optimizer = Adam(
                self.model.parameters(), lr=self.hyperparams.get("lr", 1e-4)
            )
        else:
            raise ValueError(f"Unrecognized optimizer: {optimizer}")

        # configure lr scheduler
        # scheduler = ReduceLROnPlateau(
        #     optimizer, patience=self.hyperparams.get("lr_schedule_patience", 5)
        # )
        # scheduler = torch.optim.lr_scheduler.OneCycleLR(
        #     optimizer,
        #     max_lr=self.hyperparams.get("lr", 1e-4),
        #     steps_per_epoch=num_steps_per_epoch,
        #     epochs=num_epochs,
        # )
        # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        #     optimizer, T_max=num_epochs, eta_min=0
        # )
        scheduler = LinearWarmupCosineAnnealingLR(
            optimizer,
            warmup_epochs=50,
            max_epochs=num_epochs,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

### Callbacks

In [ ]:
callbacks = [TQDMProgressBar(refresh_rate=5)]

## Model

* Fourier Feature Networks (FFN)
* Siren
* Modulated Siren (ModSiren)
* Multiplicative Filter Networks (MFN)
    * Fourier
    * Gabor


In [ ]:
from inr4ssh._src.models.models_factory import model_factory

In [ ]:
dim_in = x_init.shape[1]
dim_out = y_init.shape[1]


# args.ffn.encoder = "positional"
net = model_factory(args.model, dim_in, dim_out, model_config)
net

In [ ]:
# dim_in = x_init.shape[1]
# dim_hidden = 256
# dim_out = y_init.shape[1]
# num_layers = 5
# activation = "swish"  # Swish()  # nn.ReLU()#
# final_activation = "identity"

# mlp_net = MLP(
#     dim_in=dim_in,
#     dim_hidden=dim_hidden,
#     dim_out=dim_out,
#     num_layers=num_layers,
#     activation=get_activation(activation),
#     final_activation=get_activation(final_activation),
# )

In [ ]:
learn = ImageModel(net, learning_rate=learning_rate)

In [ ]:
out = learn.forward(x_init)

# assert out.shape[0] == x_init.shape[0]

In [ ]:
trainer = Trainer(
    min_epochs=1,
    max_epochs=num_epochs,
    accelerator="mps",
    # devices=1,
    enable_progress_bar=True,
    logger=wandb_logger,
    callbacks=callbacks,
    # gradient_clip_val=1.0,
    # gradient_clip_algorithm="norm",
)

In [ ]:
trainer.fit(
    learn,
    datamodule=dm,
)

In [ ]:
res = trainer.test(learn, dataloaders=dm.test_dataloader())

results["mlp"] = res

In [ ]:
table = [
    [
        key,
        f"{results[key][0]['test_loss']:4.4f}",
        # "{:,}".format(sum([np.prod(p.shape) for p in flow_dict[key]["model"].parameters()]))
    ]
    for key in results
]
display(
    HTML(
        tabulate.tabulate(
            table,
            tablefmt="html",
            headers=[
                "Model",
                "MSE",  # "Num Parameters"
            ],
        )
    )
)

In [ ]:
# t0 = time.time()
predictions = trainer.predict(learn, dataloaders=dm, return_predictions=True)
predictions = torch.cat(predictions)
# t1 = time.time() - t0

In [ ]:
ds_pred = dm.create_predictions_ds(predictions)
ds_pred

In [ ]:
ds_pred.pred.thin(time=2).plot.imshow(
    col="time",
    robust=True,
    col_wrap=4,
    cmap="viridis",
)

In [ ]:
ds_pred.true.thin(time=2).plot.imshow(
    col="time",
    robust=True,
    col_wrap=4,
    cmap="viridis",
)

In [ ]:
ds_pred.pred.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

In [ ]:
ds_pred.true.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")